In [1]:
import re
import nltk
import numpy as np     
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pickle as pkl
from textblob import TextBlob
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split ,GridSearchCV , cross_val_score ,RandomizedSearchCV ,KFold
from sklearn.tree import DecisionTreeClassifier ,plot_tree, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler , LabelEncoder ,MinMaxScaler,PolynomialFeatures ,MinMaxScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, HuberRegressor,LogisticRegression
from sklearn.neighbors import KNeighborsClassifier , KNeighborsRegressor
from sklearn.metrics import accuracy_score  ,mean_squared_error  ,  confusion_matrix  ,  r2_score ,classification_report, roc_curve, auc ,ConfusionMatrixDisplay
from sklearn.naive_bayes import GaussianNB ,MultinomialNB,BernoulliNB, ComplementNB
from sklearn.ensemble import RandomForestClassifier ,RandomForestRegressor , GradientBoostingClassifier
from sklearn.metrics.pairwise import linear_kernel
from sklearn.svm import SVC ,LinearSVC,SVR
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout1D, BatchNormalization
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ziad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ziad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ziad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


____________

In [2]:
df = pd.read_csv('health_sample_transformed.csv')
pd.set_option('display.max_columns',len(df.columns))
df.rename(columns=lambda x: x.strip().lower(), inplace=True)
df

,diseases,symptoms
0,panic disorder,"anxiety and nervousness, shortness of breath, ..."
1,panic disorder,"shortness of breath, depressive or psychotic s..."
2,panic disorder,"anxiety and nervousness, depression, shortness..."
3,panic disorder,"anxiety and nervousness, depressive or psychot..."
4,panic disorder,"anxiety and nervousness, depression, insomnia,..."
...,...,...
246940,open wound of the nose,"diminished hearing, facial pain, nosebleed"
246941,open wound of the nose,"diminished hearing, nosebleed"
246942,open wound of the nose,"diminished hearing, nosebleed"
246943,open wound of the nose,"diminished hearing, headache, nosebleed"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246945 entries, 0 to 246944
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   diseases  246945 non-null  object
 1   symptoms  246945 non-null  object
dtypes: object(2)
memory usage: 3.8+ MB


In [4]:
df.describe(include="all")

,diseases,symptoms
count,246945,246945
unique,773,169888
top,cystitis,"shortness of breath, sharp chest pain"
freq,1219,30


In [5]:
df.duplicated().sum()

57298

In [6]:
df.drop_duplicates(inplace=True)
df.shape

(189647, 2)

In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def remove_stopwords(text):
    words = word_tokenize(text)  
    return ' '.join([word for word in words if word.lower() not in stop_words])

def clean_text(text):
    return re.sub(r'[^a-zA-Z\s]', '', text.lower()).strip()  

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text)])  

def preprocess_text(text):
    text = clean_text(text)  
    text = remove_stopwords(text)  
    return lemmatize_text(text)  

df['symptoms'] = df['symptoms'].apply(preprocess_text)
df

,diseases,symptoms
0,panic disorder,anxiety nervousness shortness breath depressiv...
1,panic disorder,shortness breath depressive psychotic symptom ...
2,panic disorder,anxiety nervousness depression shortness breat...
3,panic disorder,anxiety nervousness depressive psychotic sympt...
4,panic disorder,anxiety nervousness depression insomnia abnorm...
...,...,...
246937,open wound of the nose,diminished hearing headache
246938,open wound of the nose,diminished hearing headache facial pain nosebleed
246941,open wound of the nose,diminished hearing nosebleed
246943,open wound of the nose,diminished hearing headache nosebleed


In [8]:
X = df['symptoms']
y = df['diseases']

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_raw)

In [10]:
pkl.dump(label_encoder, open('label_encoder.pkl', 'wb'))

In [11]:
X_train_final, X_val_raw, y_train_final_encoded, y_val_encoded = train_test_split(
    X_train_raw, y_train_encoded, test_size=0.2, random_state=42
)

In [12]:
# Filter out test samples with unseen labels
seen_mask = y_test_raw.isin(label_encoder.classes_)
X_test_seen = X_test_raw[seen_mask]
y_test_seen = y_test_raw[seen_mask]

y_test_encoded = label_encoder.transform(y_test_seen)

In [ ]:
multi_nb_clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', MultinomialNB())
])

multi_nb_clf.fit(X_train_final, y_train_final_encoded)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('clf', MultinomialNB())])

In [13]:
y_train_pred = multi_nb_clf.predict(X_train_final)
y_val_pred = multi_nb_clf.predict(X_val_raw)    

print("Training Accuracy:", accuracy_score(y_train_final_encoded, y_train_pred))
print("Validation Accuracy:", accuracy_score(y_val_encoded, y_val_pred))
print("Training Classification Report:\n", classification_report(y_train_final_encoded, y_train_pred))

Training Accuracy: 0.7518146540004779
Validation Accuracy: 0.7569865541787504
Training Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.03      0.06        31
           1       0.94      0.78      0.85       163
           2       1.00      0.24      0.39        96
           3       0.00      0.00      0.00         4
           4       0.91      0.18      0.30       112
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00        17
           8       1.00      0.57      0.72       182
           9       0.91      0.68      0.78       506
          10       0.86      0.91      0.88       762
          11       0.56      0.88      0.68       764
          12       0.77      0.41      0.53       521
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        94
          15       0.81 

In [ ]:
# Evaluate the model on the test set
y_test_pred = multi_nb_clf.predict(X_test_seen)

print("Test Accuracy:", accuracy_score(y_test_encoded, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test_encoded, y_test_pred))

Test Accuracy: 0.7500527481801877
Test Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.20      0.33         5
           1       1.00      0.90      0.95        49
           2       1.00      0.24      0.39        29
           3       0.00      0.00      0.00         1
           4       1.00      0.08      0.15        25
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         4
           8       1.00      0.63      0.78        82
           9       0.96      0.69      0.80       155
          10       0.88      0.95      0.91       238
          11       0.58      0.87      0.70       247
          12       0.80      0.49      0.61       166
          14       0.00      0.00      0.00        30
          15       0.79      0.93      0.85       168
          16       0.89      0.44      0.58       108
          17      

In [16]:
pkl.dump(multi_nb_clf, open('multi_nb_model.pkl', 'wb'))

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ComplementNB())
])

param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__max_features': [5000, 10000],
    'tfidf__min_df': [1, 3, 5],
    'tfidf__max_df': [0.9, 0.95, 1.0],
    'clf__alpha': [0.1, 0.5, 1.0]
}

grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
grid.fit(X_train_final, y_train_final_encoded)

print("Best parameters:", grid.best_params_)
print("Best cross-validation accuracy:", grid.best_score_)

complement_nb_clf = grid.best_estimator_

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'clf__alpha': 0.1, 'tfidf__max_df': 0.9, 'tfidf__max_features': 5000, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2)}
Best cross-validation accuracy: 0.6791049182854536


In [22]:
y_train_pred = complement_nb_clf.predict(X_train_final)
y_val_pred = complement_nb_clf.predict(X_val_raw)

print("Training Accuracy:", accuracy_score(y_train_final_encoded, y_train_pred))
print("Validation Accuracy:", accuracy_score(y_val_encoded, y_val_pred))
print("Training Classification Report:\n", classification_report(y_train_final_encoded, y_train_pred))

Training Accuracy: 0.6877888822060919
Validation Accuracy: 0.6823754284207751
Training Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.77      0.74        31
           1       0.90      0.91      0.91       163
           2       1.00      0.09      0.17        96
           3       1.00      0.50      0.67         4
           4       0.00      0.00      0.00       112
           5       0.00      0.00      0.00         4
           6       1.00      0.71      0.83         7
           7       0.00      0.00      0.00        17
           8       0.98      0.57      0.72       182
           9       0.84      0.66      0.74       506
          10       0.73      0.94      0.82       762
          11       0.51      0.88      0.65       764
          12       0.81      0.37      0.51       521
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        94
          15       0.78 

In [ ]:
y_test_pred = complement_nb_clf.predict(X_test_seen)

print("Test Accuracy:", accuracy_score(y_test_encoded, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test_encoded, y_test_pred))

Test Accuracy: 0.6812163730351303
Test Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.40      0.36         5
           1       0.93      0.88      0.91        49
           2       1.00      0.14      0.24        29
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        25
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         4
           8       0.98      0.63      0.77        82
           9       0.88      0.65      0.74       155
          10       0.73      0.97      0.83       238
          11       0.54      0.88      0.67       247
          12       0.82      0.48      0.61       166
          14       0.00      0.00      0.00        30
          15       0.75      0.92      0.83       168
          16       0.76      0.39      0.52       108
          17      

In [27]:
pkl.dump(complement_nb_clf, open('complement_nb_model.pkl', 'wb'))

In [ ]:
random_forest_clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])

random_forest_clf.fit(X_train_final, y_train_final_encoded)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('clf', RandomForestClassifier(random_state=42))])

In [30]:
y_train_pred = random_forest_clf.predict(X_train_final)
y_val_pred = random_forest_clf.predict(X_val_raw)

print("Training Accuracy:", accuracy_score(y_train_final_encoded, y_train_pred))
print("Validation Accuracy:", accuracy_score(y_val_encoded, y_val_pred))
print("Training Classification Report:\n", classification_report(y_train_final_encoded, y_train_pred))

Training Accuracy: 0.9223056198660328
Validation Accuracy: 0.7843395728974426
Training Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97        31
           1       0.99      0.96      0.98       163
           2       0.82      0.88      0.85        96
           3       1.00      0.75      0.86         4
           4       0.87      0.91      0.89       112
           5       1.00      0.25      0.40         4
           6       1.00      0.86      0.92         7
           7       0.77      0.59      0.67        17
           8       0.77      0.75      0.76       182
           9       0.83      0.85      0.84       506
          10       0.96      0.97      0.97       762
          11       0.91      0.90      0.91       764
          12       0.85      0.86      0.86       521
          13       1.00      0.50      0.67         2
          14       0.63      0.59      0.61        94
          15       0.97 

In [31]:
y_test_pred = random_forest_clf.predict(X_test_seen)

print("Test Accuracy:", accuracy_score(y_test_encoded, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test_encoded, y_test_pred))

Test Accuracy: 0.7838379575904632
Test Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.98      0.94      0.96        49
           2       0.66      0.72      0.69        29
           3       1.00      1.00      1.00         1
           4       0.59      0.76      0.67        25
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         1
           7       1.00      0.25      0.40         4
           8       0.80      0.63      0.71        82
           9       0.65      0.66      0.66       155
          10       0.91      0.91      0.91       238
          11       0.76      0.66      0.71       247
          12       0.56      0.66      0.61       166
          13       0.00      0.00      0.00         0
          14       0.22      0.17      0.19        30
          15       0.91      0.92      0.91       168
          16      

In [ ]:
pkl.dump(random_forest_clf, open('random_forest_model.pkl', 'wb'))

In [33]:
mlp_clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', MLPClassifier(random_state=42, max_iter=1000))  
])

mlp_clf.fit(X_train_final, y_train_final_encoded)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('clf', MLPClassifier(max_iter=1000, random_state=42))])

In [34]:
y_train_pred = mlp_clf.predict(X_train_final)
y_val_pred = mlp_clf.predict(X_val_raw)

print("Training Accuracy:", accuracy_score(y_train_final_encoded, y_train_pred))
print("Validation Accuracy:", accuracy_score(y_val_encoded, y_val_pred))
print("Training Classification Report:\n", classification_report(y_train_final_encoded, y_train_pred))

Training Accuracy: 0.8692954775773855
Validation Accuracy: 0.8189757447930398
Training Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.95        31
           1       0.96      0.99      0.98       163
           2       0.83      0.86      0.85        96
           3       0.80      1.00      0.89         4
           4       0.78      0.93      0.85       112
           5       0.38      0.75      0.50         4
           6       1.00      0.86      0.92         7
           7       0.67      0.59      0.62        17
           8       0.41      0.92      0.57       182
           9       0.82      0.74      0.78       506
          10       0.97      0.91      0.94       762
          11       0.93      0.71      0.80       764
          12       0.79      0.62      0.69       521
          13       0.33      1.00      0.50         2
          14       0.58      0.45      0.50        94
          15       0.98 

In [35]:
y_test_pred = mlp_clf.predict(X_test_seen)

print("Test Accuracy:", accuracy_score(y_test_encoded, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test_encoded, y_test_pred))

Test Accuracy: 0.8197594682983437
Test Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.80      0.73         5
           1       0.96      0.94      0.95        49
           2       0.63      0.76      0.69        29
           3       1.00      1.00      1.00         1
           4       0.64      0.84      0.72        25
           5       0.50      0.50      0.50         2
           6       1.00      1.00      1.00         1
           7       0.33      0.25      0.29         4
           8       0.52      0.89      0.65        82
           9       0.79      0.74      0.76       155
          10       0.94      0.92      0.93       238
          11       0.90      0.65      0.75       247
          12       0.79      0.64      0.71       166
          13       0.00      0.00      0.00         0
          14       0.28      0.23      0.25        30
          15       0.95      0.92      0.94       168
          16      

In [36]:
pkl.dump(mlp_clf, open('mlp_model.pkl', 'wb'))

In [37]:
log_reg_clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

log_reg_clf.fit(X_train_final, y_train_final_encoded)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('clf', LogisticRegression(max_iter=1000, random_state=42))])

In [38]:
y_train_pred = log_reg_clf.predict(X_train_final)
y_val_pred = log_reg_clf.predict(X_val_raw)

print("Training Accuracy:", accuracy_score(y_train_final_encoded, y_train_pred))
print("Validation Accuracy:", accuracy_score(y_val_encoded, y_val_pred))
print("Training Classification Report:\n", classification_report(y_train_final_encoded, y_train_pred))

Training Accuracy: 0.8286521714055021
Validation Accuracy: 0.817459794358028
Training Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.84      0.88        31
           1       0.96      0.90      0.93       163
           2       0.88      0.79      0.84        96
           3       0.00      0.00      0.00         4
           4       0.88      0.76      0.81       112
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00        17
           8       0.92      0.57      0.70       182
           9       0.92      0.68      0.78       506
          10       0.92      0.90      0.91       762
          11       0.84      0.71      0.77       764
          12       0.62      0.74      0.68       521
          13       0.00      0.00      0.00         2
          14       0.94      0.17      0.29        94
          15       0.92  

In [39]:
y_test_pred = log_reg_clf.predict(X_test_seen)

print("Test Accuracy:", accuracy_score(y_test_encoded, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test_encoded, y_test_pred))

Test Accuracy: 0.818704504694588
Test Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       0.96      0.94      0.95        49
           2       0.76      0.76      0.76        29
           3       0.00      0.00      0.00         1
           4       0.72      0.84      0.78        25
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         4
           8       0.96      0.65      0.77        82
           9       0.96      0.69      0.80       155
          10       0.90      0.93      0.91       238
          11       0.87      0.64      0.74       247
          12       0.65      0.83      0.73       166
          14       1.00      0.13      0.24        30
          15       0.92      0.93      0.92       168
          16       0.67      0.81      0.74       108
          17       

In [40]:
pkl.dump(log_reg_clf, open('log_reg_model.pkl', 'wb'))

In [13]:
# Tokenize the text
max_words = 5000
max_len = 50

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_final)

X_train_seq = tokenizer.texts_to_sequences(X_train_final)
X_val_seq = tokenizer.texts_to_sequences(X_val_raw)
X_test_seq = tokenizer.texts_to_sequences(X_test_raw)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

num_classes = len(label_encoder.classes_)

In [14]:
pkl.dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [14]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=256, input_length=max_len),
    SpatialDropout1D(0.3),
    Bidirectional(LSTM(128, return_sequences=True)),
    BatchNormalization(),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train_final_encoded,
    validation_data=(X_val_pad, y_val_encoded),
    epochs=10,
    batch_size=256,
    callbacks=[early_stop]
)

Epoch 1/10
475/475 ━━━━━━━━━━━━━━━━━━━━ 156s 320ms/step - accuracy: 0.0759 - loss: 5.0747 - val_accuracy: 0.4617 - val_loss: 2.2659
Epoch 2/10
475/475 ━━━━━━━━━━━━━━━━━━━━ 147s 309ms/step - accuracy: 0.3523 - loss: 2.4982 - val_accuracy: 0.6388 - val_loss: 1.3418
Epoch 3/10
475/475 ━━━━━━━━━━━━━━━━━━━━ 149s 313ms/step - accuracy: 0.4624 - loss: 1.9547 - val_accuracy: 0.6867 - val_loss: 1.0890
Epoch 4/10
475/475 ━━━━━━━━━━━━━━━━━━━━ 156s 328ms/step - accuracy: 0.5222 - loss: 1.6951 - val_accuracy: 0.7221 - val_loss: 0.9326
Epoch 5/10
475/475 ━━━━━━━━━━━━━━━━━━━━ 170s 359ms/step - accuracy: 0.5594 - loss: 1.5385 - val_accuracy: 0.7491 - val_loss: 0.8507
Epoch 6/10
475/475 ━━━━━━━━━━━━━━━━━━━━ 168s 354ms/step - accuracy: 0.5839 - loss: 1.4412 - val_accuracy: 0.7577 - val_loss: 0.8062
Epoch 7/10
475/475 ━━━━━━━━━━━━━━━━━━━━ 163s 344ms/step - accuracy: 0.6029 - loss: 1.3601 - val_accuracy: 0.7644 - val_loss: 0.7401
Epoch 8/10
475/475 ━━━━━━━━━━━━━━━━━━━━ 157s 331ms/step - accuracy: 0.6201 -

In [15]:
train_loss, train_accuracy = model.evaluate(X_train_pad, y_train_final_encoded, verbose=0)
val_loss, val_accuracy = model.evaluate(X_val_pad, y_val_encoded, verbose=0)

print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

y_val_pred = np.argmax(model.predict(X_val_pad), axis=1)
print("Validation Classification Report:\n", classification_report(y_val_encoded, y_val_pred))

Train Loss: 0.5941, Train Accuracy: 0.7991
Validation Loss: 0.6430, Validation Accuracy: 0.7870
949/949 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.96      0.92      0.94        53
           2       0.93      0.54      0.68        26
           4       0.87      0.74      0.80        27
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         4
           8       0.92      0.57      0.71        42
           9       0.90      0.68      0.78       147
          10       1.00      0.90      0.95       204
          11       0.89      0.73      0.80       202
          12       0.44      0.74      0.55       125
          14       0.80      0.17      0.28        24
          15       0.89      0.95      0.92       130
          16   

In [20]:
# Only evaluate on test samples with seen labels
X_test_seen_seq = tokenizer.texts_to_sequences(X_test_seen)
X_test_seen_pad = pad_sequences(X_test_seen_seq, maxlen=max_len, padding='post')

y_test_pred = np.argmax(model.predict(X_test_seen_pad), axis=1)

print("Test Accuracy:", accuracy_score(y_test_encoded, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test_encoded, y_test_pred))

1185/1185 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step
Test Accuracy: 0.7839962021310265
Test Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.40      0.57         5
           1       0.98      0.92      0.95        49
           2       1.00      0.52      0.68        29
           3       0.00      0.00      0.00         1
           4       0.69      0.80      0.74        25
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         4
           8       0.98      0.63      0.77        82
           9       0.92      0.69      0.79       155
          10       0.98      0.91      0.94       238
          11       0.90      0.64      0.75       247
          12       0.43      0.70      0.53       166
          14       1.00      0.13      0.24        30
          15       0.94      0.95      0.94       168
          16       0.59    

In [21]:
pkl.dump(model, open('lstm_model.pkl', 'wb'))